# Title and Introduction
# Exploratory Data Analysis and Preprocessing for Complaint Analysis

This notebook performs exploratory data analysis (EDA) and preprocessing on the Consumer Financial Protection Bureau (CFPB) complaint dataset, as per Task 1 requirements. The goal is to understand the data's structure, content, and quality, and prepare it for the Retrieval-Augmented Generation (RAG) pipeline. The analysis aligns with CrediTrust's business objectives (e.g., reducing trend identification time) and ensures the data is suitable for semantic search and embedding.

# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from pathlib import Path

%matplotlib inline
sns.set_style('whitegrid')

In [ ]:
# Set up paths
DATA_DIR = Path("../data")
DATA_DIR.mkdir(exist_ok=True)
dataset_path = "../data/raw/complaints.csv"

CHUNK_SIZE = 10000 
# Initialize variables for chunked analysis
total_rows = 0
product_counts = pd.Series(dtype=int)
missing_narratives = 0
narrative_lengths = []
chunks = []

# Process dataset in chunks
for chunk in pd.read_csv(dataset_path, chunksize=CHUNK_SIZE, low_memory=False):
    total_rows += len(chunk)
    # Count products
    chunk_product_counts = chunk['Product'].value_counts()
    product_counts = product_counts.add(chunk_product_counts, fill_value=0)
    # Count missing narratives
    missing_narratives += chunk['Consumer complaint narrative'].isnull().sum()
    # Calculate narrative lengths
    chunk['narrative_length'] = chunk['Consumer complaint narrative'].apply(
        lambda x: len(str(x).split()) if pd.notnull(x) else 0
    )
    narrative_lengths.extend(chunk['narrative_length'].tolist())
    # Store chunk for preprocessing
    chunks.append(chunk)

print(f"Loaded dataset with {total_rows} rows in chunks of {CHUNK_SIZE}.")
df_head = chunks[0].head()  # Display first few rows of first chunk
df_head

In [3]:
# Business context: Define target products for filtering
TARGET_PRODUCTS = [
    "Credit card",
    "Personal loan",
    "Buy Now, Pay Later (BNPL)",
    "Savings account",
    "Money transfers"
]

In [ ]:
def load_data(dataset_path):
    """Load the CFPB dataset and return a DataFrame."""
    try:
        df = pd.read_csv(dataset_path)
        print(f"Loaded dataset with {len(df)} records.")
        return df
    except FileNotFoundError:
        print("Dataset file not found. Please provide the correct path.")
        return None

dataset_path = "../data/raw/complaints.csv"
df = load_data(dataset_path)

In [5]:
def analyze_complaint_distribution(df):
    """Analyze and visualize complaint distribution across products."""
    product_counts = df['Product'].value_counts()
    print("\nComplaint Distribution by Product:")
    print(product_counts)
    
    # Visualize distribution
    plt.figure(figsize=(10, 6))
    sns.barplot(x=product_counts.values, y=product_counts.index)
    plt.title("Complaint Distribution by Product")
    plt.xlabel("Number of Complaints")
    plt.ylabel("Product")
    plt.tight_layout()
    plt.savefig(DATA_DIR / "product_distribution.png")
    plt.close()
    
    return product_counts

In [6]:
def analyze_narrative_lengths(df):
    """Calculate and visualize word counts of consumer complaint narratives."""
    df['narrative_length'] = df['Consumer complaint narrative'].apply(
        lambda x: len(str(x).split()) if pd.notnull(x) else 0
    )
    
    print("\nNarrative Length Statistics:")
    print(df['narrative_length'].describe())
    
    # Visualize length distribution
    plt.figure(figsize=(10, 6))
    sns.histplot(df['narrative_length'], bins=50, kde=True)
    plt.title("Distribution of Complaint Narrative Word Counts")
    plt.xlabel("Word Count")
    plt.ylabel("Frequency")
    plt.tight_layout()
    plt.savefig(DATA_DIR / "narrative_length_distribution.png")
    plt.close()
    
    return df['narrative_length'].describe()

In [7]:
def analyze_missing_narratives(df):
    """Identify complaints with and without narratives."""
    missing_narratives = df[' veritable complaint narrative'].isnull().sum()
    non_missing_narratives = len(df) - missing_narratives
    print(f"\nMissing Narratives: {missing_narratives} ({missing_narratives/len(df)*100:.2f}%)")
    print(f"Non-Missing Narratives: {non_missing_narratives} ({non_missing_narratives/len(df)*100:.2f}%)")
    
    return missing_narratives, non_missing_narratives

In [8]:
def clean_narrative(text):
    """Clean complaint narratives for RAG suitability."""
    if pd.isnull(text):
        return text
    # Lowercase
    text = text.lower()
    # Remove special characters, keep alphanumeric and basic punctuation
    text = re.sub(r'[^a-z0-9\s.,!?]', '', text)
    # Remove boilerplate phrases
    boilerplate_phrases = [
        r"i am writing to file a complaint",
        r"please assist me",
        r"this is regarding my account"
    ]
    for phrase in boilerplate_phrases:
        text = re.sub(phrase, '', text, flags=re.IGNORECASE)
    # Remove excessive whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [9]:
def preprocess_data(df):
    """Filter and clean the dataset for the five target products."""
    # Filter for target products
    initial_count = len(df)
    df_filtered = df[df['Product'].isin(TARGET_PRODUCTS)].copy()
    print(f"\nFiltered dataset to {len(df_filtered)} records from {initial_count} for target products.")
    
    # Remove empty narratives
    non_empty_count = len(df_filtered)
    df_filtered = df_filtered[df_filtered['Consumer complaint narrative'].notnull()]
    print(f"Removed {non_empty_count - len(df_filtered)} records with empty narratives.")
    
    # Clean narratives
    df_filtered['cleaned_narrative'] = df_filtered['Consumer complaint narrative'].apply(clean_narrative)
    
    # Verify no empty cleaned narratives
    empty_cleaned = df_filtered['cleaned_narrative'].isnull().sum()
    if empty_cleaned > 0:
        print(f"Warning: {empty_cleaned} cleaned narratives are empty.")
    
    return df_filtered

In [10]:
def save_cleaned_data(df, output_path):
    """Save the cleaned and filtered dataset."""
    df.to_csv(output_path, index=False)
    print(f"Saved cleaned dataset to {output_path}")

In [11]:
def main():
    # Load dataset (replace with actual CFPB dataset path)
    dataset_path = "../data/raw/complaints.csv"  # Update with actual path
    df = load_data(dataset_path)
    if df is None:
        return
    
    # Business-focused EDA
    print("Performing business-focused EDA...")
    product_counts = analyze_complaint_distribution(df)
    narrative_stats = analyze_narrative_lengths(df)
    missing_narratives, non_missing_narratives = analyze_missing_narratives(df)
    
    # Preprocess data for RAG
    print("\nPreprocessing data for RAG pipeline...")
    df_filtered = preprocess_data(df)
    
    # Save cleaned dataset
    save_cleaned_data(df_filtered, OUTPUT_PATH)
    
    # Generate EDA summary for report
    summary = f"""
### EDA Summary

**Business Insights**:
The CFPB dataset contains {len(df)} complaints, with {product_counts.sum()} across all products. The distribution of complaints acrossAmenities across the five target products (Credit Card, Personal Loan, BNPL, Savings Account, Money Transfers) is as follows:
{', '.join([f'{k}: {v}' for k, v in product_counts.items() if k in TARGET_PRODUCTS])}.
This distribution highlights key areas of customer dissatisfaction, particularly for high-volume products like {product_counts.idxmax()} ({product_counts.max()} complaints), which aligns with CrediTrust’s need to quickly identify major complaint trends (KPI 1). For example, a high volume of BNPL complaints could indicate operational issues like billing disputes or fraud, critical for Product Managers like Asha to address proactively.

**RAG Suitability**:
The narrative length analysis shows a mean word count of {narrative_stats['mean']:.2f} (min: {narrative_stats['min']:.0f}, max: {narrative_stats['max']:.0f}). {f'Many narratives are short (<50 words), which may limit semantic richness for RAG retrieval.' if narrative_stats['mean'] < 50 else 'Most narratives are sufficiently long for effective embedding.'} {missing_narratives} complaints ({missing_narratives/len(df)*100:.2f}%) lack narratives, reducing the dataset’s effective size for RAG. The cleaning process removed boilerplate text and special characters to improve embedding quality, ensuring semantic coherence for the vector store.

**Next Steps**:
The filtered dataset ({len(df_filtered)} records) is saved to {OUTPUT_PATH}, ready for chunking and embedding in Task 2. The focus on high-volume products like BNPL will guide the creation of test questions for Task 3, ensuring the RAG system addresses key stakeholder needs.
"""
    with open(DATA_DIR / "eda_summary.md", "w") as f:
        f.write(summary)
    print(f"EDA summary saved to {DATA_DIR / 'eda_summary.md'}")

if __name__ == "__main__":
    main()

Dataset file not found. Please provide the correct path.
